In [1]:
import pandas as pd
import csv
import requests
import datetime
import json

CSV_URL = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
csvData = []
countriesISOCode = []
countriesWithISOCode = []
caseValues = []

# datei wird gefetch von der seite als csv und wird in eine liste umgewandelt
def fetchData(arr):
    with requests.Session() as s:
        download = s.get(CSV_URL)
        decodedContent = download.content.decode('utf-8')
        cr = csv.reader(decodedContent.splitlines(), delimiter=',')
        return list(cr)

# daten werden auf eine sieben tage periode gefiltert
def adjustListToDayPeriod(days,csvData):
    results = []
    startDate = datetime.date.today()
    sevenDaysBefore = startDate - datetime.timedelta(days=days)
    # filterungsschleife
    for idx in range(1,len(csvData)):
        if(csvData[idx][0] not in countriesISOCode):
            countriesISOCode.append(csvData[idx][0])
        csvDate = datetime.datetime.strptime(csvData[idx][3], '%Y-%m-%d').date()
        if((sevenDaysBefore<=csvDate<=startDate) == True):
           results.append(csvData[idx])
    return results

def getCSVData(csvData, iso_code):
    results = []
    avg_seven_day_cases = 0
    total_cases = 0
    population = 0

    # korrespondierende daten finden 
    for i in range(len(csvData)):
        if len(results) == 7:
            break
        elif iso_code != csvData[i][0]:
            continue
        elif iso_code == csvData[i][0]:
            results.append(csvData[i])
            if csvData[i][5]:
                avg_seven_day_cases += (int(float(csvData[i][5])))

    # total cases extrahieren
    if len(results) > 0 and results[len(results)-1][4]:
        total_cases = int(float(results[len(results)-1][4]))

    # population extrahieren
    if len(results) > 0 and results[len(results)-1][44]:
        population = int(float(results[len(results)-1][44]))
    
    # string formatierung auf tausender stellen
    total_cases = f'{total_cases:,}'.replace(",", ".")
    population = f'{population:,}'.replace(",", ".")
    avg_seven_day_cases = f'{avg_seven_day_cases:,}'.replace(",", ".")

    # datensammlung um die reichtweite der legende zu bestimmen
    # wird nur genutzt um eine übersicht zu bekommen
    # if int(avg_seven_day_cases.replace(".", "")) not in caseValues:
    #     caseValues.append(int(avg_seven_day_cases.replace(".", "")))

    return total_cases, avg_seven_day_cases, population

# anfertigung der JSON datei welches dann ein modifizierte json datei ausgibt
def modifyOutputJSON():
    with open("../data/countries.geo.json") as jsonData:
        data = json.load(jsonData)
        for feature in data["features"]:
            name = feature["properties"]["ADMIN"]
            iso_code = feature["properties"]["ISO_A3"]
            del feature["properties"]
            total_cases, avg_seven_day_cases, population = getCSVData(csvData,iso_code)
            # anpassung der properties von der json datei
            feature["properties"]= {
                "name": name,
                "total_cases": total_cases,
                "avg_seven_day_cases": avg_seven_day_cases ,
                "population": population,
                "iso_code": iso_code
            }
            countriesWithISOCode.append({"value":iso_code, "country": name})

        # ausgabe der neue geojson datei
        with open("../web/countries.geo.json", "w+") as countries:
            countries.write(json.dumps(data))

        # ausgabe der datenliste für die searchbox
        with open("../web/countries.json", "w+") as countries:
            countries.write(json.dumps({"countriesWithISOCode":countriesWithISOCode}))

        # diese funktion ist nur dazu da um eine übersicht über die werte der letzten sieben tage zu bekommen um
        # eine skala zu schaffen
        # with open("../range.json", "w+") as reichweite:
        #     caseValues.sort()
        #     reichweite.write(json.dumps({"values":caseValues}))

# ausführen der steps
csvData = fetchData(csvData)
csvData = adjustListToDayPeriod(7, csvData)
modifyOutputJSON()